### input itp, output tcl for drawing the elastic network

In [ ]:
import re

def extract_numbers_from_itp(itp_file):
    start_marker = "; Rubber band"
    end_marker = "; Short elastic bonds for extended regions"
    extracting = False
    extracted_data = []

    with open(itp_file, 'r') as file:
        for line in file:
            if start_marker in line:
                extracting = True
                continue
            elif end_marker in line:
                break
            
            if extracting:
                numbers = re.findall(r'\d+', line)
                if len(numbers) >= 2:
                    num1, num2 = int(numbers[0]) - 1, int(numbers[1]) - 1
                    extracted_data.append(f"{{{num1} {num2}}}")
    
    return extracted_data

def generate_tcl_file(input_file, output_file):
    bonds = extract_numbers_from_itp(input_file)
    bond_list_content = " ".join(bonds)
    
    tcl_content = f"""
set bond_list {{
{bond_list_content}
}}

# Loop through all bonds
foreach bond $bond_list {{
    set sel1 [atomselect top "index [lindex $bond 0]"]
    set sel2 [atomselect top "index [lindex $bond 1]"]

    # Draw a bond as a line
    graphics top line [lindex [$sel1 get {{x y z}}] 0] [lindex [$sel2 get {{x y z}}] 0] width 3
}}
"""
    
    with open(output_file, 'w') as file:
        file.write(tcl_content)
    print(f"TCL file '{output_file}' generated successfully.")

if __name__ == "__main__":
    itp_file = "molecule_0.itp"  # Replace with the actual ITP filename
    output_tcl = "draw_elastic_network.tcl" # need to change output TCL file name if using different input ITP
    generate_tcl_file(itp_file, output_tcl)
